本书到目前为止一直都在使用命令式编程，**它使用编程语句改变程序状态**。考虑下面这段简单的命令式程序。

In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

fancy_func(1, 2, 3, 4) # 10

10

和我们预期的一样，在运行语句`e = add(a, b)`时，Python会做加法运算并将结果存储在变量`e`中，从而令程序的状态发生改变。类似地，后面的两条语句`f = add(c, d)`和`g = add(e, f)`会依次做加法运算并存储变量。

虽然使用命令式编程很方便，**但它的运行可能很慢**。一方面，即使`fancy_func`函数中的`add`是被重复调用的函数，Python也会逐一执行这3条函数调用语句。另一方面，**我们需要保存变量`e`和`f`的值直到`fancy_func`中所有语句执行结束**。这是因为在执行`e = add(a, b)`和`f = add(c, d)`这2条语句之后我们并不知道变量`e`和`f`是否会被程序的其他部分使用。

与命令式编程不同，**符号式编程通常在计算流程完全定义好后才被执行**。多个深度学习框架，如**Theano和TensorFlow，都使用了符号式编程**。通常，符号式编程的程序需要下面3个步骤：

1. 定义计算流程；
2. 把计算流程编译成可执行的程序；
3. 给定输入，调用编译好的程序执行。

下面我们用符号式编程重新实现本节开头给出的命令式编程代码。

In [2]:
def add_str():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_str():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_str():
    return add_str() + fancy_func_str() + '''
print(fancy_func(1, 2, 3, 4))
'''

prog = evoke_str()


In [3]:
print(prog)



def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))



In [4]:
y = compile(prog, '', 'exec')
exec(y)

10


**以上定义的3个函数都仅以字符串的形式返回计算流程**。最后，我们**通过`compile`函数编译完整的计算流程并运行**。**由于在编译时系统能够完整地获取整个程序，因此有更多空间优化计算**。例如，编译的时候可以将程序改写成`print((1 + 2) + (3 + 4))`，甚至直接改写成`print(10)`。这样不仅**减少了函数调用**，还**节省了内存**。

对比这两种编程方式，我们可以看到以下两点。

* 命令式编程更方便。当我们在Python里使用命令式编程时，大部分代码编写起来都很直观。同时，命令式编程更容易调试。这是因为我们可以很方便地获取并打印所有的中间变量值，或者使用Python的调试工具。

* **符号式编程更高效并更容易移植**。一方面，在编译的时候系统容易做更多优化；另一方面，**符号式编程可以将程序变成一个与Python无关的格式，从而可以使程序在非Python环境下运行，以避开Python解释器的性能问题**。

## 8.1.1 混合式编程取两者之长

大部分深度学习框架在命令式编程和符号式编程之间二选一。例如，Theano和受其启发的后来者`TensorFlow1.x`使用了符号式编程，`Chainer`和它的追随者**`PyTorch`使用了命令式编程**。开发人员在设计`Tensorflow2.x`时思考了这个问题：有没有可能既得到命令式编程的好处，又享受符号式编程的优势？开发者们认为，用户应该用纯命令式编程进行开发和调试；**当需要产品级别的计算性能和部署时，用户可以将大部分命令式程序转换成符号式程序来运行**。Tensorflow通过提供静态图转换器`tf.function`,实现对两种编程方式的支持。在 Tensorflow

在不使用静态图转换器`tf.function`时，用户编写的`python`函数默认会采用命令式编程逐行执行，符合`python`编程的直觉，便于调试，但因为框架不能获得完整的静态运算图，不能进行优化，且动态图不能序列化。官方由此推出了静态图转换器**`tf.function`，其作用在`python_function`后会将这个函数"编译"成一个运算图，接受`input_tensors`为输入并用图执行的方式计算结果，可以加速函数执行，并且可以被序列化后供任何其他语言（如C++和Java）调用**，这样用户只需要在使用动态图运算编写和测试完毕函数之后使用tf.function装饰一下就能够获得静态图的所有优点。

In [21]:
import tensorflow as tf
import numpy as np
from nose.tools import assert_raises

## 8.1.2 tf.function 的使用

### 8.1.2.1 基础

`tf.function`可以定义一个`Tensorflow`操作，既可以命令式的执行它，

In [6]:
@tf.function
def add(a, b):
    return a+b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

也可以在图中执行它，并求其梯度，

In [7]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

也可以定义嵌套定义（当然，在实际使用中，可以直接在顶层定义，会自动对子图进行转换），

In [9]:
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

### 8.1.2.2 追踪与多态

`Python` 的**动态类型意味着用户可以传递多种类型的参数**，这也许会导致函数产生不同的行为。

在另一方面，`Tensorflow`的静态图需要确定的数据类型和维度。**`tf.function` 通过`retracing`函数调用来弥补这一差距**，并在必要的时候产生正确的计算图。`tf.function`大多数微妙的行为都产生自`retracing`行为。

我们可以用不同的参数调用同一函数来观察`retracing`行为：

In [12]:
# Functions are polymorphic

@tf.function
def double(a):
  print("Tracing with", a)   # 跟踪输入变量类型？
  return a + a

print(double(tf.constant(1)))
print()   # 打印空行
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



如果希望控制 `tracing` 行为，可以用如下方式操作：

* 创建新的 `tf.function`。**分离 `tf.fucntion` 对象，保证没有共享的计算图引用**。
* 使用 `get_concrete_function` 方法，得到特定的计算图。
* 声明 `input_signature` 当调用 `tf.function` 时，仅跟踪与输入签名一致的调用。

**注：通过例子来理解。**

In [20]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print()
print("Executing traced function")
print()
print(double_strings(tf.constant("a")))
print()
print(double_strings(a=tf.constant("b")))

print("Using a concrete trace with incompatible types will throw an error")
print()
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Obtaining concrete trace

Executing traced function

tf.Tensor(b'aa', shape=(), dtype=string)

tf.Tensor(b'bb', shape=(), dtype=string)
Using a concrete trace with incompatible types will throw an error



In [23]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
# We specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)


### 8.1.2.3 追踪触发的时机

多态函数 `tf.function` 会缓存之前追踪行为触发生成过的具体函数。缓存的键由传入的参数确定，对于 `tf.Tensor` 参数而言，是其维度和类型，而对于 `Python` 元语，是其值。对于其它 Python类型，使用对象 id，即对每个不同的类实例都会触发独立的追踪行为，并生成相应的静态图。